In [1]:
import keras

2024-05-03 13:47:46.851282: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 13:47:46.851792: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-03 13:47:46.857423: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-03 13:47:47.097984: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 13:47:48.305815: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
keras.__version__

'3.3.3'

In [3]:
import tensorflow as tf
tf.__version__

'2.16.1'

## Example of taking feature extractor from siamese network

In [4]:
# Model creation
def _create_model(nneurons, nfilters, ndropout, npool):
    # 4 layers
    inputs = keras.Input((400, 400, 3))
    
    x = keras.layers.Conv2D(nneurons[0], (nfilters[0], nfilters[0]), padding="same", activation="relu")(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size=(npool[0], npool[0]), data_format='channels_last')(x)

    x = keras.layers.Conv2D(nneurons[1], (nfilters[1], nfilters[1]), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size=(npool[1], npool[1]), data_format='channels_last')(x)

    x = keras.layers.Conv2D(nneurons[2], (nfilters[2], nfilters[2]), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size=(npool[2], npool[2]), data_format='channels_last')(x)

    x = keras.layers.Conv2D(nneurons[3], (nfilters[3], nfilters[3]), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size=(npool[3], npool[3]), data_format='channels_last')(x)
    x = keras.layers.Dropout(ndropout[0])(x)

    pooledOutput = keras.layers.GlobalAveragePooling2D()(x)
    pooledOutput = keras.layers.Dense(nneurons[4])(pooledOutput)
    outputs = keras.layers.Dense(nneurons[5])(pooledOutput)

    model = keras.Model(inputs, outputs)
    return model

keras.saving.get_custom_objects().clear()

@keras.saving.register_keras_serializable(package="MyLayers")
class euclidean_lambda(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(euclidean_lambda, self).__init__(**kwargs)
        self.name = 'euclidean_lambda'

    def call(self, featA, featB):
        squared = keras.ops.square(featA-featB)
        return squared
    

def siamese_model(nneurons, nfilters, ndropout, npool):
    feature_extractor_model = _create_model(nneurons, nfilters, ndropout, npool)
    imgA = keras.Input(shape=(400, 400, 3))
    imgB = keras.Input(shape=(400, 400, 3))
    featA = feature_extractor_model(imgA)
    featB = feature_extractor_model(imgB)
    distance = euclidean_lambda()(featA, featB)
    # distance = keras.layers.Lambda(_euclidean_distance, output_shape=(128,))([featA, featB])
    # distance = keras.layers.Lambda(lambda vec: keras.ops.square(vec[0]-vec[1]), output_shape=(128,))([featA, featB])
    # distance = keras.layers.Lambda(_euclidean_distance)([featA, featB])
    outputs = keras.layers.Dense(1, activation="sigmoid")(distance)
    model = keras.Model(inputs=[imgA, imgB], outputs=outputs)
    return model

In [5]:
# Create siamese model
nneurons = [32, 64, 64, 128, 256, 128]
nfilters = [3, 3, 5, 5]
ndropout = [0.4]
npool = [4, 4, 4, 4]
siamesemodel = siamese_model(nneurons, nfilters, ndropout, npool)

In [6]:
# load weights - note that this includes 2 additional weights for the output layer 
siamesemodel.load_weights('./optimized_weights.h5')

In [7]:
siamesemodel.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 400, 400,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 400, 400,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 128)       │    393,856 │ input_layer_1[0]… │
│ (Functional)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ euclidean_lambda    │ (None, 128)       │          0 │ functional_1[0][… │
│ (euclidean_lambda)  │                   │            │ functional_1[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        129 │ euclidean_lambda… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 393,985 (1.50 MB)

 Trainable params: 393,409 (1.50 MB)

 Non-trainable params: 576 (2.25 KB)

In [8]:
# Save siamese model
keras.saving.save_model(siamesemodel, './siamese_model.keras', overwrite=True)

In [9]:
keras.saving.get_custom_objects()

{'MyLayers>euclidean_lambda': __main__.euclidean_lambda}

In [10]:
siamese_model2 = keras.models.load_model(
    "siamese_model.keras",
    custom_objects = keras.saving.get_custom_objects()
)

In [11]:
siamese_model2.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 400, 400,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 400, 400,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 128)       │    393,856 │ input_layer_1[0]… │
│ (Functional)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ euclidean_lambda    │ (None, 128)       │          0 │ functional_1[0][… │
│ (euclidean_lambda)  │                   │            │ functional_1[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        129 │ euclidean_lambda… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 393,985 (1.50 MB)

 Trainable params: 393,409 (1.50 MB)

 Non-trainable params: 576 (2.25 KB)